In [1]:
import sys
# import sklearn
from sklearn.metrics import precision_score, recall_score, roc_auc_score, f1_score, confusion_matrix, roc_curve, auc
import os
import pandas as pd
import pickle
import tensorflow as tf
import numpy as np
import time
import math
import scipy.io as sio

from keras.layers import Conv2D, Dense, Flatten, ELU, BatchNormalization, LSTMCell, StackedRNNCells,\
    RNN, Permute, Dropout, Concatenate, Input, concatenate, Lambda, Reshape, Lambda

from keras import backend as K
from keras.utils import plot_model
from keras.models import Sequential, Model
from keras.layers.merge import add
from keras.callbacks import TensorBoard

from keras import callbacks

from keras import regularizers

from keras.callbacks import ModelCheckpoint

import datetime

from keras.models import load_model
import matplotlib.pyplot as plt



Using TensorFlow backend.


In [1]:
!nvidia-smi

Sat Nov 09 17:17:15 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.97       Driver Version: 440.97       CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 166... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   36C    P3     9W /  N/A |    153MiB /  6144MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
data_files=[]
for i in range(1,33):
    if(1<=i<=9):
        s='s0'+str(i)
    else:
        s='s'+str(i)
    data_files.append(s)
print(len(data_files),data_files)

# emotions=['arousal','valence']

#baseline_preprocessing=['yes','no']
emotions=['arousal']

baseline_preprocessing=['yes']

32 ['s01', 's02', 's03', 's04', 's05', 's06', 's07', 's08', 's09', 's10', 's11', 's12', 's13', 's14', 's15', 's16', 's17', 's18', 's19', 's20', 's21', 's22', 's23', 's24', 's25', 's26', 's27', 's28', 's29', 's30', 's31', 's32']


In [3]:
# data_files=[data_files[0]]
data_files,emotions,baseline_preprocessing

(['s01',
  's02',
  's03',
  's04',
  's05',
  's06',
  's07',
  's08',
  's09',
  's10',
  's11',
  's12',
  's13',
  's14',
  's15',
  's16',
  's17',
  's18',
  's19',
  's20',
  's21',
  's22',
  's23',
  's24',
  's25',
  's26',
  's27',
  's28',
  's29',
  's30',
  's31',
  's32'],
 ['arousal'],
 ['yes'])

In [4]:
def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)
        

# Example
createFolder('./data/dd')

In [5]:
import gc

In [6]:
gc.collect()

265

In [5]:
np.random.seed(32)

window_size = 128

cnn_suffix ='.mat_win_128_cnn_dataset.pkl'
rnn_suffix ='.mat_win_128_rnn_dataset.pkl'
label_suffix ='.mat_win_128_labels.pkl'

for with_or_without in baseline_preprocessing:
    for arousal_or_valence in emotions:
        for data_file in data_files:
            
            print('---------------------------------------------------')
            print(with_or_without+' '+arousal_or_valence+' '+data_file)
            print('---------------------------------------------------\n')

            #data_file    ='s17'
            #arousal_or_valence = 'valence'
            #with_or_without = 'yes'

            dataset_dir = 'deap_shuffled_data/'+with_or_without+'_'+arousal_or_valence+'/'
            ###load training set
            with open(dataset_dir + data_file + cnn_suffix, "rb") as fp:
                cnn_datasets = pickle.load(fp)
            with open(dataset_dir + data_file + rnn_suffix, "rb") as fp:
                rnn_datasets = pickle.load(fp)
            with open(dataset_dir + data_file + label_suffix, "rb") as fp:
                labels = pickle.load(fp)
                labels = np.transpose(labels)
                print("loaded shape:",labels.shape)
            lables_backup = labels

            print(cnn_datasets.shape)
            print(rnn_datasets.shape)
            print(labels.shape)

            #print("cnn_dataset shape before reshape:", np.shape(cnn_datasets))
            # cnn_datasets = cnn_datasets.reshape(len(cnn_datasets), window_size, 9,9, 1)
            #print("cnn_dataset shape after reshape:", np.shape(cnn_datasets))
            one_hot_labels = np.array(list(pd.get_dummies(labels)))

            labels = np.asarray(pd.get_dummies(labels), dtype=np.int8)

            print(labels.shape)
            # shuffle data
            index = np.array(range(0, len(labels)))
            np.random.shuffle(index)

            cnn_datasets   = cnn_datasets[index]
            rnn_datasets   = rnn_datasets[index]
            labels  = labels[index]

            print(cnn_datasets.shape)
            print(rnn_datasets.shape)
            print(labels.shape)

            #print("**********(" + time.asctime(time.localtime(time.time())) + ") Load and Split dataset End **********\n")
            #print("**********(" + time.asctime(time.localtime(time.time())) + ") Define parameters and functions Begin: **********\n")
            print('cnn_datasets.shape,rnn_datasets.shape,labels.shape : ',cnn_datasets.shape,rnn_datasets.shape,labels.shape)
            #important
            cnn_datasets=cnn_datasets.reshape(2400,9,9,-1)# imp
            print(cnn_datasets.shape)
            print('cnn_datasets.shape : ',cnn_datasets.shape)







            print('========================Train / Test Shapes==============================')



            fold=10
            curr_fold=0
            #for curr_fold in range(fold): # kernel dies
                
            #   print('curr_fold / fold : ',curr_fold,' / ',fold)


            max_acc_acc_list=[]

            max_acc_loss_list=[]


            min_loss_acc_list=[]

            min_loss_loss_list=[]

            fold_size = cnn_datasets.shape[0]//fold
            indexes_list = [i for i in range(len(cnn_datasets))]
            indexes = np.array(indexes_list)
            split_list = [i for i in range(curr_fold*fold_size,(curr_fold+1)*fold_size)]
            split = np.array(split_list)

            cnn_test = cnn_datasets[split] 
            labels_test = labels[split]
            rnn_test = rnn_datasets[split]

            split = np.array(list(set(indexes_list)^set(split_list)))

            cnn_train = cnn_datasets[split]
            rnn_train = rnn_datasets[split]
            labels_train = labels[split]

            # train_sample = labels_train.shape[0]
            # print("training examples:", train_sample)
            # test_sample = labels_test.shape[0]
            # print("test examples    :",test_sample)
            print(cnn_test.shape,rnn_test.shape,cnn_train.shape,rnn_train.shape)

            print('================================ DNN ============================================')

            input_cnn=Input(shape=(9,9,128))

            print('input_cnn: ',input_cnn.shape)

            conv1=Conv2D(32,
                          kernel_size=(4,4),
                          strides=(1,1),
                          padding='same',
                          input_shape=(9,9,128)
                         )(input_cnn)


            bn1=BatchNormalization()(conv1)

            elu1=ELU()(bn1)

            print('elu1: ',elu1.shape)
            #?,9,9,32
            conv2=Conv2D(64,
                          kernel_size=(4,4),
                          strides=(1,1),
                          padding='same'
                         )(elu1)

            bn2=BatchNormalization()(conv2)

            elu2=ELU()(bn2)

            print('elu2: ',elu2.shape)
            #?,9,9,64

            conv3=Conv2D(128,
                          kernel_size=(4,4),
                          strides=(1,1),
                          padding='same'
                         )(elu2)

            bn3=BatchNormalization()(conv3)

            elu3=ELU()(bn3)


            print('elu3: ',elu3.shape)
            #?,9,9,128

            # mc.add(Flatten())
            # mc.add(Lambda(lambda x:x,output_shape=(9,9,32*4*128)))
            # mc.add(Lambda(K.reshape((-1,9,9,32*4*128))))
            reshape1=Reshape((9,9,-1))(elu3)


            print('reshape1: ',reshape1.shape)
            #?,9,9,32*4*128

            conv4=Conv2D(13,#32*4*128,
                          kernel_size=(1,1),
                          strides=(1,1),
                          padding='same'
                         )(reshape1)

            bn4=BatchNormalization()(conv4)

            elu4=ELU()(bn4)


            print('elu4: ',elu4.shape)
            #?,9,9,13 #32*4*128

            # mc.add(Flatten())

            # mc.add(Lambda(lambda x:x,output_shape=([13*9*9])))
            # mc.add(Lambda(K.reshape((None,13*9*9))))
            reshape2=Reshape(([13*9*9]))(elu4)


            print('reshape2: ',reshape2.shape)




            cnn_out_fuse=reshape2


            # cube=K.reshape(e3,(-1,9,9,32*4*128))#(e3)


            # flat=K.reshape(e4,(-1,13*9*9))#(e4) #1053

            # rnn_in=K.placeholder(shape=(None,128,32))
            # rnn_in=tf.convert_to_tensor(rnn_datasets,dtype='float32')


            # rnn_in.get_shape().as_list()

            # rnn_in=K.placeholder(shape=(None,128,32))
            # rnn_in_flat=K.reshape(rnn_in,[-1,32])

            # print('rnn_in ',rnn_in.shape)

            input_rnn=Input(shape=(128,32))
            print('input_rnn',input_rnn.shape)

            rnn_in_flat=Reshape((-1,32))(input_rnn)
            print('rnn_in_flat',rnn_in_flat.shape)
            # rnn_in_flat = Lambda(lambda x:x[:,0])(input_rnn)

            # rnn_fc_in1 =Dense(32)(rnn_in_flat)
            rnn_fc_in1 =Dense(1024)(rnn_in_flat)
            rnn_fc_in=ELU()(rnn_fc_in1)
            print('rnn_fc_in',rnn_fc_in.shape)

            # rnn_fc_in =Dense(1024)(input_rnn)

            # lstm_in=Reshape((-1,128,1024))(rnn_fc_in)
            lstm_in=Reshape((-1,1024))(rnn_fc_in)
            print('lstm_in',lstm_in.shape)

            cells=[]

            for i in range(2):
                cell=LSTMCell(32,unit_forget_bias=True,dropout=0.5)#'forget_bias'=1.0,'state_is_tuple'=True
                cells.append(cell)
            #     print(cell.shape)

            # lstm_cell=StackedRNNCells(cells)
            lstm_cell=RNN(cells)(lstm_in)
            # print(lstm_cell.shape)
            # op,states=RNN(cells)(lstm_in)
            print('lstm_cell',lstm_cell.shape)
            # output=K.transpose_shape((1,0,2),lstm_cell)
            # output=Permute((1,0,2))(lstm_cell)
            # output.reshape()
            output=lstm_cell
            print('output',output.shape)
            rnn_output=output[-1]
            # rnn_output

            print('rnn_output',rnn_output.shape)
            # shape_rnn_out=rnn_output.get_shape().as_list()
            lstm_fc_out=Dense(1024)(output)#shape_rnn_out[1]

            print('lstm_fc_out',lstm_fc_out.shape)


            # lstm_fc_out_2=Dense(1053)(lstm_fc_out)#shape_rnn_out[1]



            lstm_fc_drop=Dropout(0.5)(lstm_fc_out)
            # lstm_fc_drop
            print('lstm_fc_drop',lstm_fc_drop.shape)

            # fuse_cnn_rnn=add([cnn_out_fuse,lstm_fc_drop])

            fuse_cnn_rnn=concatenate([cnn_out_fuse,lstm_fc_drop])
            print('fuse_cnn_rnn ',fuse_cnn_rnn.shape)
            y=Dense(2,activation='softmax')(fuse_cnn_rnn) ## ,activity_regularizer=regularizers.l2(0.5)
            print(y.shape)
            y_pred=K.argmax(y,1)
            # y_pred=K.argmax(K.softmax(y))
            # y_posi=K.softmax(y)
            print('y ',y)


            directory_le007a='./lightningedge007a_results/'+with_or_without+'/'+arousal_or_valence+'/'+data_file
            try:
                if not os.path.exists(directory_le007a):
                    os.makedirs(directory_le007a)
            except OSError:
                print ('Error: Creating directory. ' +  directory_le007a)



            model=Model(inputs=[input_cnn,input_rnn],outputs=y)
            model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

            print('Model : ',with_or_without+'_'+arousal_or_valence+'_'+data_file+'_'+str(curr_fold)+'_fold')
            print(model.summary())

            m_val_acc=ModelCheckpoint('lightningedge007a_results/'+with_or_without+'/'+arousal_or_valence+'/'+data_file+'/'+'max_acc_'+with_or_without+'_'+arousal_or_valence+'_'+data_file+'.h5',monitor='val_accuracy',mode='max',verbose=1,save_best_only=True)
            m_val_loss=ModelCheckpoint('lightningedge007a_results/'+with_or_without+'/'+arousal_or_valence+'/'+data_file+'/'+'min_loss_'+with_or_without+'_'+arousal_or_valence+'_'+data_file+'.h5',monitor='val_loss',mode='min',verbose=1,save_best_only=True)


            tb_log_dir='lightningedge007a_results\\'+with_or_without+'\\'+arousal_or_valence+'\\'+data_file+'\\'+'logs_'+with_or_without+'_'+arousal_or_valence+'_'+data_file
            #tb_log_dir='lightningedge007a_results/'+with_or_without+'/'+arousal_or_valence+'/'+data_file+'/'+'logs_'+with_or_without+'_'+arousal_or_valence+'_'+data_file

            createFolder(tb_log_dir)


            #log_dir='lightningedge007a_results\\'+with_or_without+'\\'+arousal_or_valence+'\\'+data_file+'\\'+'logs_'+with_or_without+'_'+arousal_or_valence+'_'+data_file+'_'+str(curr_fold)+'_fold\\' #datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
            tensorboard_callback = callbacks.TensorBoard(log_dir=tb_log_dir)#, histogram_freq=1)

            plotpicture=plot_model(model, to_file='lightningedge007a_results/'+with_or_without+'/'+arousal_or_valence+'/'+data_file+'/'+'model_'+with_or_without+'_'+arousal_or_valence+'_'+data_file+'.png', show_shapes=True)



            history=model.fit([cnn_train,rnn_train],labels_train,batch_size=128,epochs=50,callbacks=[tensorboard_callback,m_val_acc,m_val_loss],validation_split=0.2)


            #Plot values
            plt.plot(history.history['accuracy'])
            plt.plot(history.history['val_accuracy'])
            plt.title('max_acc_'+with_or_without+'_'+arousal_or_valence+'_'+data_file)
            plt.ylabel('Accuracy')
            plt.xlabel('Epoch')
            plt.legend(['Train', 'Validation'], loc='upper right',bbox_to_anchor=(1.3,1))
            plt.savefig('lightningedge007a_results/'+with_or_without+'/'+arousal_or_valence+'/'+data_file+'/'+'max_acc_'+with_or_without+'_'+arousal_or_valence+'_'+data_file+'.png',bbox_inches='tight')
            #plt.show()
            plt.close()


            # Plot training & validation loss values
            plt.plot(history.history['loss'])
            plt.plot(history.history['val_loss'])
            plt.title('min_loss_'+with_or_without+'_'+arousal_or_valence+'_'+data_file+'_'+str(curr_fold)+'_fold')
            plt.ylabel('Loss')
            plt.xlabel('Epoch')
            plt.legend(['Train', 'Validation'], loc='upper right',bbox_to_anchor=(1.3,1))
            plt.savefig('lightningedge007a_results/'+with_or_without+'/'+arousal_or_valence+'/'+data_file+'/'+'min_loss_'+with_or_without+'_'+arousal_or_valence+'_'+data_file+'.png',bbox_inches='tight')
            #plt.show()
            plt.close()

            vam=load_model('lightningedge007a_results/'+with_or_without+'/'+arousal_or_valence+'/'+data_file+'/'+'max_acc_'+with_or_without+'_'+arousal_or_valence+'_'+data_file+'.h5')
            vlm=load_model('lightningedge007a_results/'+with_or_without+'/'+arousal_or_valence+'/'+data_file+'/'+'min_loss_'+with_or_without+'_'+arousal_or_valence+'_'+data_file+'.h5')
            # pred_labels=vam()
            vam_eval=vam.evaluate([cnn_test,rnn_test],labels_test)
            print('max acc: ',vam_eval)
            vlm_eval=vlm.evaluate([cnn_test,rnn_test],labels_test)
            print('min loss: ',vlm_eval)


            max_acc_acc_list.append(vam_eval[1])

            max_acc_loss_list.append(vam_eval[0])


            min_loss_acc_list.append(vlm_eval[1])

            min_loss_loss_list.append(vlm_eval[0])

            pickle.dump(max_acc_acc_list,open('lightningedge007a_results/'+with_or_without+'/'+arousal_or_valence+'/'+data_file+'/'+'max_acc_acc_list_'+with_or_without+'_'+arousal_or_valence+'_'+data_file+'.pkl','wb'))

            pickle.dump(max_acc_loss_list,open('lightningedge007a_results/'+with_or_without+'/'+arousal_or_valence+'/'+data_file+'/'+'max_acc_loss_list_'+with_or_without+'_'+arousal_or_valence+'_'+data_file+'.pkl','wb'))


            pickle.dump(min_loss_acc_list,open('lightningedge007a_results/'+with_or_without+'/'+arousal_or_valence+'/'+data_file+'/'+'min_loss_acc_list_'+with_or_without+'_'+arousal_or_valence+'_'+data_file+'.pkl','wb'))

            pickle.dump(min_loss_loss_list,open('lightningedge007a_results/'+with_or_without+'/'+arousal_or_valence+'/'+data_file+'/'+'min_loss_loss_list_'+with_or_without+'_'+arousal_or_valence+'_'+data_file+'.pkl','wb'))



---------------------------------------------------
yes arousal s01
---------------------------------------------------

loaded shape: (2400,)
(2400, 128, 9, 9)
(2400, 128, 32)
(2400,)
(2400, 2)
(2400, 128, 9, 9)
(2400, 128, 32)
(2400, 2)
cnn_datasets.shape,rnn_datasets.shape,labels.shape :  (2400, 128, 9, 9) (2400, 128, 32) (2400, 2)
(2400, 9, 9, 128)
cnn_datasets.shape :  (2400, 9, 9, 128)
========================Train / Test Shapes==============================
(240, 9, 9, 128) (240, 128, 32) (2160, 9, 9, 128) (2160, 128, 32)
================================ DNN ============================================
input_cnn:  (None, 9, 9, 128)
elu1:  (None, 9, 9, 32)
elu2:  (None, 9, 9, 64)
elu3:  (None, 9, 9, 128)
reshape1:  (None, 9, 9, None)
elu4:  (None, 9, 9, 13)
reshape2:  (None, 1053)
input_rnn (None, 128, 32)
rnn_in_flat (None, None, 32)
rnn_fc_in (None, 128, 1024)
lstm_in (None, None, 1024)
lstm_cell (None, 32)
output (None, 32)
rnn_output (32,)
lstm_fc_out (None, 1024)
lstm_fc_dro

C:\Users\ABHISHEK_VERMA\Anaconda3\lib\site-packages\keras\callbacks\callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (9.941433). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


1728/1728 [==============================] - 26s 15ms/step - loss: 0.2985 - accuracy: 0.8640 - val_loss: 0.1483 - val_accuracy: 0.9537

Epoch 00001: val_accuracy improved from -inf to 0.95370, saving model to lightningedge007a_results/yes/arousal/s01/max_acc_yes_arousal_s01.h5

Epoch 00001: val_loss improved from inf to 0.14827, saving model to lightningedge007a_results/yes/arousal/s01/min_loss_yes_arousal_s01.h5
Epoch 2/50
1728/1728 [==============================] - 3s 2ms/step - loss: 0.0293 - accuracy: 0.9931 - val_loss: 0.0682 - val_accuracy: 0.9769

Epoch 00002: val_accuracy improved from 0.95370 to 0.97685, saving model to lightningedge007a_results/yes/arousal/s01/max_acc_yes_arousal_s01.h5

Epoch 00002: val_loss improved from 0.14827 to 0.06817, saving model to lightningedge007a_results/yes/arousal/s01/min_loss_yes_arousal_s01.h5
Epoch 3/50
1728/1728 [==============================] - 3s 2ms/step - loss: 0.0081 - accuracy: 0.9994 - val_loss: 0.0359 - val_accuracy: 0.9931

Epoch


Epoch 00023: val_accuracy did not improve from 1.00000

Epoch 00023: val_loss improved from 0.01005 to 0.00982, saving model to lightningedge007a_results/yes/arousal/s01/min_loss_yes_arousal_s01.h5
Epoch 24/50
1728/1728 [==============================] - 3s 2ms/step - loss: 1.3123e-04 - accuracy: 1.0000 - val_loss: 0.0095 - val_accuracy: 1.0000

Epoch 00024: val_accuracy did not improve from 1.00000

Epoch 00024: val_loss improved from 0.00982 to 0.00952, saving model to lightningedge007a_results/yes/arousal/s01/min_loss_yes_arousal_s01.h5
Epoch 25/50
1728/1728 [==============================] - 3s 2ms/step - loss: 1.2525e-04 - accuracy: 1.0000 - val_loss: 0.0093 - val_accuracy: 1.0000

Epoch 00025: val_accuracy did not improve from 1.00000

Epoch 00025: val_loss improved from 0.00952 to 0.00934, saving model to lightningedge007a_results/yes/arousal/s01/min_loss_yes_arousal_s01.h5
Epoch 26/50
1728/1728 [==============================] - 3s 2ms/step - loss: 1.1768e-04 - accuracy: 1.000

1728/1728 [==============================] - 3s 2ms/step - loss: 4.5746e-05 - accuracy: 1.0000 - val_loss: 0.0070 - val_accuracy: 1.0000

Epoch 00047: val_accuracy did not improve from 1.00000

Epoch 00047: val_loss improved from 0.00703 to 0.00696, saving model to lightningedge007a_results/yes/arousal/s01/min_loss_yes_arousal_s01.h5
Epoch 48/50
1728/1728 [==============================] - 3s 2ms/step - loss: 4.4130e-05 - accuracy: 1.0000 - val_loss: 0.0069 - val_accuracy: 1.0000

Epoch 00048: val_accuracy did not improve from 1.00000

Epoch 00048: val_loss improved from 0.00696 to 0.00686, saving model to lightningedge007a_results/yes/arousal/s01/min_loss_yes_arousal_s01.h5
Epoch 49/50
1728/1728 [==============================] - 3s 2ms/step - loss: 4.2295e-05 - accuracy: 1.0000 - val_loss: 0.0069 - val_accuracy: 1.0000

Epoch 00049: val_accuracy did not improve from 1.00000

Epoch 00049: val_loss improved from 0.00686 to 0.00685, saving model to lightningedge007a_results/yes/arousal/

...


Epoch 00050: val_accuracy did not improve from 0.98843

Epoch 00050: val_loss improved from 0.03172 to 0.03170, saving model to lightningedge007a_results/yes/arousal/s03/min_loss_yes_arousal_s03.h5
240/240 [==============================] - 1s 3ms/step
max acc:  [0.00792253830525927, 0.9958333373069763]
240/240 [==============================] - 1s 3ms/step
min loss:  [0.008038910596405912, 0.9958333373069763]
---------------------------------------------------
yes arousal s04
---------------------------------------------------

loaded shape: (2400,)
(2400, 128, 9, 9)
(2400, 128, 32)
(2400,)
(2400, 2)
(2400, 128, 9, 9)
(2400, 128, 32)
(2400, 2)
cnn_datasets.shape,rnn_datasets.shape,labels.shape :  (2400, 128, 9, 9) (2400, 128, 32) (2400, 2)
(2400, 9, 9, 128)
cnn_datasets.shape :  (2400, 9, 9, 128)
========================Train / Test Shapes==============================
(240, 9, 9, 128) (240, 128, 32) (2160, 9, 9, 128) (2160, 128, 32)
================================ DNN =============

C:\Users\ABHISHEK_VERMA\Anaconda3\lib\site-packages\keras\callbacks\callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (10.531249). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


1728/1728 [==============================] - 26s 15ms/step - loss: 0.6526 - accuracy: 0.7373 - val_loss: 0.3831 - val_accuracy: 0.8356

Epoch 00001: val_accuracy improved from -inf to 0.83565, saving model to lightningedge007a_results/yes/arousal/s04/max_acc_yes_arousal_s04.h5

Epoch 00001: val_loss improved from inf to 0.38308, saving model to lightningedge007a_results/yes/arousal/s04/min_loss_yes_arousal_s04.h5
Epoch 2/50
1728/1728 [==============================] - 3s 2ms/step - loss: 0.1983 - accuracy: 0.9207 - val_loss: 0.3599 - val_accuracy: 0.8565

Epoch 00002: val_accuracy improved from 0.83565 to 0.85648, saving model to lightningedge007a_results/yes/arousal/s04/max_acc_yes_arousal_s04.h5

Epoch 00002: val_loss improved from 0.38308 to 0.35995, saving model to lightningedge007a_results/yes/arousal/s04/min_loss_yes_arousal_s04.h5
Epoch 3/50
1728/1728 [==============================] - 3s 2ms/step - loss: 0.1069 - accuracy: 0.9688 - val_loss: 0.2494 - val_accuracy: 0.8866

Epoch


Epoch 00026: val_accuracy did not improve from 0.93750

Epoch 00026: val_loss did not improve from 0.16136
Epoch 27/50
1728/1728 [==============================] - 3s 2ms/step - loss: 4.1825e-04 - accuracy: 1.0000 - val_loss: 0.1743 - val_accuracy: 0.9352

Epoch 00027: val_accuracy did not improve from 0.93750

Epoch 00027: val_loss did not improve from 0.16136
Epoch 28/50
1728/1728 [==============================] - 3s 2ms/step - loss: 3.9295e-04 - accuracy: 1.0000 - val_loss: 0.1736 - val_accuracy: 0.9375

Epoch 00028: val_accuracy did not improve from 0.93750

Epoch 00028: val_loss did not improve from 0.16136
Epoch 29/50
1728/1728 [==============================] - 3s 2ms/step - loss: 3.6690e-04 - accuracy: 1.0000 - val_loss: 0.1738 - val_accuracy: 0.9375

Epoch 00029: val_accuracy did not improve from 0.93750

Epoch 00029: val_loss did not improve from 0.16136
Epoch 30/50
1728/1728 [==============================] - 3s 2ms/step - loss: 3.4130e-04 - accuracy: 1.0000 - val_loss: 0.

KeyboardInterrupt: 

In [6]:
data_files=data_files[4:]

In [7]:
data_files

['s05',
 's06',
 's07',
 's08',
 's09',
 's10',
 's11',
 's12',
 's13',
 's14',
 's15',
 's16',
 's17',
 's18',
 's19',
 's20',
 's21',
 's22',
 's23',
 's24',
 's25',
 's26',
 's27',
 's28',
 's29',
 's30',
 's31',
 's32']

In [8]:
np.random.seed(32)

window_size = 128

cnn_suffix ='.mat_win_128_cnn_dataset.pkl'
rnn_suffix ='.mat_win_128_rnn_dataset.pkl'
label_suffix ='.mat_win_128_labels.pkl'

for with_or_without in baseline_preprocessing:
    for arousal_or_valence in emotions:
        for data_file in data_files:
            
            print('---------------------------------------------------')
            print(with_or_without+' '+arousal_or_valence+' '+data_file)
            print('---------------------------------------------------\n')

            #data_file    ='s17'
            #arousal_or_valence = 'valence'
            #with_or_without = 'yes'

            dataset_dir = 'deap_shuffled_data/'+with_or_without+'_'+arousal_or_valence+'/'
            ###load training set
            with open(dataset_dir + data_file + cnn_suffix, "rb") as fp:
                cnn_datasets = pickle.load(fp)
            with open(dataset_dir + data_file + rnn_suffix, "rb") as fp:
                rnn_datasets = pickle.load(fp)
            with open(dataset_dir + data_file + label_suffix, "rb") as fp:
                labels = pickle.load(fp)
                labels = np.transpose(labels)
                print("loaded shape:",labels.shape)
            lables_backup = labels

            print(cnn_datasets.shape)
            print(rnn_datasets.shape)
            print(labels.shape)

            #print("cnn_dataset shape before reshape:", np.shape(cnn_datasets))
            # cnn_datasets = cnn_datasets.reshape(len(cnn_datasets), window_size, 9,9, 1)
            #print("cnn_dataset shape after reshape:", np.shape(cnn_datasets))
            one_hot_labels = np.array(list(pd.get_dummies(labels)))

            labels = np.asarray(pd.get_dummies(labels), dtype=np.int8)

            print(labels.shape)
            # shuffle data
            index = np.array(range(0, len(labels)))
            np.random.shuffle(index)

            cnn_datasets   = cnn_datasets[index]
            rnn_datasets   = rnn_datasets[index]
            labels  = labels[index]

            print(cnn_datasets.shape)
            print(rnn_datasets.shape)
            print(labels.shape)

            #print("**********(" + time.asctime(time.localtime(time.time())) + ") Load and Split dataset End **********\n")
            #print("**********(" + time.asctime(time.localtime(time.time())) + ") Define parameters and functions Begin: **********\n")
            print('cnn_datasets.shape,rnn_datasets.shape,labels.shape : ',cnn_datasets.shape,rnn_datasets.shape,labels.shape)
            #important
            cnn_datasets=cnn_datasets.reshape(2400,9,9,-1)# imp
            print(cnn_datasets.shape)
            print('cnn_datasets.shape : ',cnn_datasets.shape)







            print('========================Train / Test Shapes==============================')



            fold=10
            curr_fold=0
            #for curr_fold in range(fold): # kernel dies
                
            #   print('curr_fold / fold : ',curr_fold,' / ',fold)


            max_acc_acc_list=[]

            max_acc_loss_list=[]


            min_loss_acc_list=[]

            min_loss_loss_list=[]

            fold_size = cnn_datasets.shape[0]//fold
            indexes_list = [i for i in range(len(cnn_datasets))]
            indexes = np.array(indexes_list)
            split_list = [i for i in range(curr_fold*fold_size,(curr_fold+1)*fold_size)]
            split = np.array(split_list)

            cnn_test = cnn_datasets[split] 
            labels_test = labels[split]
            rnn_test = rnn_datasets[split]

            split = np.array(list(set(indexes_list)^set(split_list)))

            cnn_train = cnn_datasets[split]
            rnn_train = rnn_datasets[split]
            labels_train = labels[split]

            # train_sample = labels_train.shape[0]
            # print("training examples:", train_sample)
            # test_sample = labels_test.shape[0]
            # print("test examples    :",test_sample)
            print(cnn_test.shape,rnn_test.shape,cnn_train.shape,rnn_train.shape)

            print('================================ DNN ============================================')

            input_cnn=Input(shape=(9,9,128))

            print('input_cnn: ',input_cnn.shape)

            conv1=Conv2D(32,
                          kernel_size=(4,4),
                          strides=(1,1),
                          padding='same',
                          input_shape=(9,9,128)
                         )(input_cnn)


            bn1=BatchNormalization()(conv1)

            elu1=ELU()(bn1)

            print('elu1: ',elu1.shape)
            #?,9,9,32
            conv2=Conv2D(64,
                          kernel_size=(4,4),
                          strides=(1,1),
                          padding='same'
                         )(elu1)

            bn2=BatchNormalization()(conv2)

            elu2=ELU()(bn2)

            print('elu2: ',elu2.shape)
            #?,9,9,64

            conv3=Conv2D(128,
                          kernel_size=(4,4),
                          strides=(1,1),
                          padding='same'
                         )(elu2)

            bn3=BatchNormalization()(conv3)

            elu3=ELU()(bn3)


            print('elu3: ',elu3.shape)
            #?,9,9,128

            # mc.add(Flatten())
            # mc.add(Lambda(lambda x:x,output_shape=(9,9,32*4*128)))
            # mc.add(Lambda(K.reshape((-1,9,9,32*4*128))))
            reshape1=Reshape((9,9,-1))(elu3)


            print('reshape1: ',reshape1.shape)
            #?,9,9,32*4*128

            conv4=Conv2D(13,#32*4*128,
                          kernel_size=(1,1),
                          strides=(1,1),
                          padding='same'
                         )(reshape1)

            bn4=BatchNormalization()(conv4)

            elu4=ELU()(bn4)


            print('elu4: ',elu4.shape)
            #?,9,9,13 #32*4*128

            # mc.add(Flatten())

            # mc.add(Lambda(lambda x:x,output_shape=([13*9*9])))
            # mc.add(Lambda(K.reshape((None,13*9*9))))
            reshape2=Reshape(([13*9*9]))(elu4)


            print('reshape2: ',reshape2.shape)




            cnn_out_fuse=reshape2


            # cube=K.reshape(e3,(-1,9,9,32*4*128))#(e3)


            # flat=K.reshape(e4,(-1,13*9*9))#(e4) #1053

            # rnn_in=K.placeholder(shape=(None,128,32))
            # rnn_in=tf.convert_to_tensor(rnn_datasets,dtype='float32')


            # rnn_in.get_shape().as_list()

            # rnn_in=K.placeholder(shape=(None,128,32))
            # rnn_in_flat=K.reshape(rnn_in,[-1,32])

            # print('rnn_in ',rnn_in.shape)

            input_rnn=Input(shape=(128,32))
            print('input_rnn',input_rnn.shape)

            rnn_in_flat=Reshape((-1,32))(input_rnn)
            print('rnn_in_flat',rnn_in_flat.shape)
            # rnn_in_flat = Lambda(lambda x:x[:,0])(input_rnn)

            # rnn_fc_in1 =Dense(32)(rnn_in_flat)
            rnn_fc_in1 =Dense(1024)(rnn_in_flat)
            rnn_fc_in=ELU()(rnn_fc_in1)
            print('rnn_fc_in',rnn_fc_in.shape)

            # rnn_fc_in =Dense(1024)(input_rnn)

            # lstm_in=Reshape((-1,128,1024))(rnn_fc_in)
            lstm_in=Reshape((-1,1024))(rnn_fc_in)
            print('lstm_in',lstm_in.shape)

            cells=[]

            for i in range(2):
                cell=LSTMCell(32,unit_forget_bias=True,dropout=0.5)#'forget_bias'=1.0,'state_is_tuple'=True
                cells.append(cell)
            #     print(cell.shape)

            # lstm_cell=StackedRNNCells(cells)
            lstm_cell=RNN(cells)(lstm_in)
            # print(lstm_cell.shape)
            # op,states=RNN(cells)(lstm_in)
            print('lstm_cell',lstm_cell.shape)
            # output=K.transpose_shape((1,0,2),lstm_cell)
            # output=Permute((1,0,2))(lstm_cell)
            # output.reshape()
            output=lstm_cell
            print('output',output.shape)
            rnn_output=output[-1]
            # rnn_output

            print('rnn_output',rnn_output.shape)
            # shape_rnn_out=rnn_output.get_shape().as_list()
            lstm_fc_out=Dense(1024)(output)#shape_rnn_out[1]

            print('lstm_fc_out',lstm_fc_out.shape)


            # lstm_fc_out_2=Dense(1053)(lstm_fc_out)#shape_rnn_out[1]



            lstm_fc_drop=Dropout(0.5)(lstm_fc_out)
            # lstm_fc_drop
            print('lstm_fc_drop',lstm_fc_drop.shape)

            # fuse_cnn_rnn=add([cnn_out_fuse,lstm_fc_drop])

            fuse_cnn_rnn=concatenate([cnn_out_fuse,lstm_fc_drop])
            print('fuse_cnn_rnn ',fuse_cnn_rnn.shape)
            y=Dense(2,activation='softmax')(fuse_cnn_rnn) ## ,activity_regularizer=regularizers.l2(0.5)
            print(y.shape)
            y_pred=K.argmax(y,1)
            # y_pred=K.argmax(K.softmax(y))
            # y_posi=K.softmax(y)
            print('y ',y)


            directory_le007a='./lightningedge007a_results/'+with_or_without+'/'+arousal_or_valence+'/'+data_file
            try:
                if not os.path.exists(directory_le007a):
                    os.makedirs(directory_le007a)
            except OSError:
                print ('Error: Creating directory. ' +  directory_le007a)



            model=Model(inputs=[input_cnn,input_rnn],outputs=y)
            model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

            print('Model : ',with_or_without+'_'+arousal_or_valence+'_'+data_file+'_'+str(curr_fold)+'_fold')
            print(model.summary())

            m_val_acc=ModelCheckpoint('lightningedge007a_results/'+with_or_without+'/'+arousal_or_valence+'/'+data_file+'/'+'max_acc_'+with_or_without+'_'+arousal_or_valence+'_'+data_file+'.h5',monitor='val_accuracy',mode='max',verbose=1,save_best_only=True)
            m_val_loss=ModelCheckpoint('lightningedge007a_results/'+with_or_without+'/'+arousal_or_valence+'/'+data_file+'/'+'min_loss_'+with_or_without+'_'+arousal_or_valence+'_'+data_file+'.h5',monitor='val_loss',mode='min',verbose=1,save_best_only=True)


            tb_log_dir='lightningedge007a_results\\'+with_or_without+'\\'+arousal_or_valence+'\\'+data_file+'\\'+'logs_'+with_or_without+'_'+arousal_or_valence+'_'+data_file
            #tb_log_dir='lightningedge007a_results/'+with_or_without+'/'+arousal_or_valence+'/'+data_file+'/'+'logs_'+with_or_without+'_'+arousal_or_valence+'_'+data_file

            createFolder(tb_log_dir)


            #log_dir='lightningedge007a_results\\'+with_or_without+'\\'+arousal_or_valence+'\\'+data_file+'\\'+'logs_'+with_or_without+'_'+arousal_or_valence+'_'+data_file+'_'+str(curr_fold)+'_fold\\' #datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
            tensorboard_callback = callbacks.TensorBoard(log_dir=tb_log_dir)#, histogram_freq=1)

            plotpicture=plot_model(model, to_file='lightningedge007a_results/'+with_or_without+'/'+arousal_or_valence+'/'+data_file+'/'+'model_'+with_or_without+'_'+arousal_or_valence+'_'+data_file+'.png', show_shapes=True)



            history=model.fit([cnn_train,rnn_train],labels_train,batch_size=128,epochs=50,callbacks=[tensorboard_callback,m_val_acc,m_val_loss],validation_split=0.2)


            #Plot values
            plt.plot(history.history['accuracy'])
            plt.plot(history.history['val_accuracy'])
            plt.title('max_acc_'+with_or_without+'_'+arousal_or_valence+'_'+data_file)
            plt.ylabel('Accuracy')
            plt.xlabel('Epoch')
            plt.legend(['Train', 'Validation'], loc='upper right',bbox_to_anchor=(1.3,1))
            plt.savefig('lightningedge007a_results/'+with_or_without+'/'+arousal_or_valence+'/'+data_file+'/'+'max_acc_'+with_or_without+'_'+arousal_or_valence+'_'+data_file+'.png',bbox_inches='tight')
            #plt.show()
            plt.close()


            # Plot training & validation loss values
            plt.plot(history.history['loss'])
            plt.plot(history.history['val_loss'])
            plt.title('min_loss_'+with_or_without+'_'+arousal_or_valence+'_'+data_file+'_'+str(curr_fold)+'_fold')
            plt.ylabel('Loss')
            plt.xlabel('Epoch')
            plt.legend(['Train', 'Validation'], loc='upper right',bbox_to_anchor=(1.3,1))
            plt.savefig('lightningedge007a_results/'+with_or_without+'/'+arousal_or_valence+'/'+data_file+'/'+'min_loss_'+with_or_without+'_'+arousal_or_valence+'_'+data_file+'.png',bbox_inches='tight')
            #plt.show()
            plt.close()

            vam=load_model('lightningedge007a_results/'+with_or_without+'/'+arousal_or_valence+'/'+data_file+'/'+'max_acc_'+with_or_without+'_'+arousal_or_valence+'_'+data_file+'.h5')
            vlm=load_model('lightningedge007a_results/'+with_or_without+'/'+arousal_or_valence+'/'+data_file+'/'+'min_loss_'+with_or_without+'_'+arousal_or_valence+'_'+data_file+'.h5')
            # pred_labels=vam()
            vam_eval=vam.evaluate([cnn_test,rnn_test],labels_test)
            print('max acc: ',vam_eval)
            vlm_eval=vlm.evaluate([cnn_test,rnn_test],labels_test)
            print('min loss: ',vlm_eval)


            max_acc_acc_list.append(vam_eval[1])

            max_acc_loss_list.append(vam_eval[0])


            min_loss_acc_list.append(vlm_eval[1])

            min_loss_loss_list.append(vlm_eval[0])

            pickle.dump(max_acc_acc_list,open('lightningedge007a_results/'+with_or_without+'/'+arousal_or_valence+'/'+data_file+'/'+'max_acc_acc_list_'+with_or_without+'_'+arousal_or_valence+'_'+data_file+'.pkl','wb'))

            pickle.dump(max_acc_loss_list,open('lightningedge007a_results/'+with_or_without+'/'+arousal_or_valence+'/'+data_file+'/'+'max_acc_loss_list_'+with_or_without+'_'+arousal_or_valence+'_'+data_file+'.pkl','wb'))


            pickle.dump(min_loss_acc_list,open('lightningedge007a_results/'+with_or_without+'/'+arousal_or_valence+'/'+data_file+'/'+'min_loss_acc_list_'+with_or_without+'_'+arousal_or_valence+'_'+data_file+'.pkl','wb'))

            pickle.dump(min_loss_loss_list,open('lightningedge007a_results/'+with_or_without+'/'+arousal_or_valence+'/'+data_file+'/'+'min_loss_loss_list_'+with_or_without+'_'+arousal_or_valence+'_'+data_file+'.pkl','wb'))



---------------------------------------------------
yes arousal s05
---------------------------------------------------

loaded shape: (2400,)
(2400, 128, 9, 9)
(2400, 128, 32)
(2400,)
(2400, 2)
(2400, 128, 9, 9)
(2400, 128, 32)
(2400, 2)
cnn_datasets.shape,rnn_datasets.shape,labels.shape :  (2400, 128, 9, 9) (2400, 128, 32) (2400, 2)
(2400, 9, 9, 128)
cnn_datasets.shape :  (2400, 9, 9, 128)
========================Train / Test Shapes==============================
(240, 9, 9, 128) (240, 128, 32) (2160, 9, 9, 128) (2160, 128, 32)
================================ DNN ============================================
input_cnn:  (None, 9, 9, 128)
elu1:  (None, 9, 9, 32)
elu2:  (None, 9, 9, 64)
elu3:  (None, 9, 9, 128)
reshape1:  (None, 9, 9, None)
elu4:  (None, 9, 9, 13)
reshape2:  (None, 1053)
input_rnn (None, 128, 32)
rnn_in_flat (None, None, 32)
rnn_fc_in (None, 128, 1024)
lstm_in (None, None, 1024)
lstm_cell (None, 32)
output (None, 32)
rnn_output (32,)
lstm_fc_out (None, 1024)
lstm_fc_dro

C:\Users\ABHISHEK_VERMA\Anaconda3\lib\site-packages\keras\callbacks\callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (10.274689). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


1728/1728 [==============================] - 26s 15ms/step - loss: 0.5042 - accuracy: 0.8119 - val_loss: 0.2353 - val_accuracy: 0.9074

Epoch 00001: val_accuracy improved from -inf to 0.90741, saving model to lightningedge007a_results/yes/arousal/s05/max_acc_yes_arousal_s05.h5

Epoch 00001: val_loss improved from inf to 0.23526, saving model to lightningedge007a_results/yes/arousal/s05/min_loss_yes_arousal_s05.h5
Epoch 2/50
1728/1728 [==============================] - 3s 2ms/step - loss: 0.1271 - accuracy: 0.9537 - val_loss: 0.1610 - val_accuracy: 0.9468

Epoch 00002: val_accuracy improved from 0.90741 to 0.94676, saving model to lightningedge007a_results/yes/arousal/s05/max_acc_yes_arousal_s05.h5

Epoch 00002: val_loss improved from 0.23526 to 0.16098, saving model to lightningedge007a_results/yes/arousal/s05/min_loss_yes_arousal_s05.h5
Epoch 3/50
1728/1728 [==============================] - 3s 2ms/step - loss: 0.0718 - accuracy: 0.9792 - val_loss: 0.1149 - val_accuracy: 0.9630

Epoch

1728/1728 [==============================] - 3s 2ms/step - loss: 2.8417e-04 - accuracy: 1.0000 - val_loss: 0.0572 - val_accuracy: 0.9745

Epoch 00027: val_accuracy did not improve from 0.97917

Epoch 00027: val_loss did not improve from 0.05560
Epoch 28/50
1728/1728 [==============================] - 3s 2ms/step - loss: 2.6610e-04 - accuracy: 1.0000 - val_loss: 0.0585 - val_accuracy: 0.9745

Epoch 00028: val_accuracy did not improve from 0.97917

Epoch 00028: val_loss did not improve from 0.05560
Epoch 29/50
1728/1728 [==============================] - 3s 2ms/step - loss: 2.4857e-04 - accuracy: 1.0000 - val_loss: 0.0590 - val_accuracy: 0.9745

Epoch 00029: val_accuracy did not improve from 0.97917

Epoch 00029: val_loss did not improve from 0.05560
Epoch 30/50
1728/1728 [==============================] - 3s 2ms/step - loss: 2.3774e-04 - accuracy: 1.0000 - val_loss: 0.0591 - val_accuracy: 0.9745

Epoch 00030: val_accuracy did not improve from 0.97917

Epoch 00030: val_loss did not impro

KeyboardInterrupt: 

In [12]:
data_file=data_files[1:1+5]

['s06', 's07', 's08', 's09', 's10']

In [7]:
data_files=data_files[5:10]
data_files

['s06', 's07', 's08', 's09', 's10']

In [8]:
np.random.seed(32)

window_size = 128

cnn_suffix ='.mat_win_128_cnn_dataset.pkl'
rnn_suffix ='.mat_win_128_rnn_dataset.pkl'
label_suffix ='.mat_win_128_labels.pkl'

for with_or_without in baseline_preprocessing:
    for arousal_or_valence in emotions:
        for data_file in data_files:
            
            print('---------------------------------------------------')
            print(with_or_without+' '+arousal_or_valence+' '+data_file)
            print('---------------------------------------------------\n')

            #data_file    ='s17'
            #arousal_or_valence = 'valence'
            #with_or_without = 'yes'

            dataset_dir = 'deap_shuffled_data/'+with_or_without+'_'+arousal_or_valence+'/'
            ###load training set
            with open(dataset_dir + data_file + cnn_suffix, "rb") as fp:
                cnn_datasets = pickle.load(fp)
            with open(dataset_dir + data_file + rnn_suffix, "rb") as fp:
                rnn_datasets = pickle.load(fp)
            with open(dataset_dir + data_file + label_suffix, "rb") as fp:
                labels = pickle.load(fp)
                labels = np.transpose(labels)
                print("loaded shape:",labels.shape)
            lables_backup = labels

            print(cnn_datasets.shape)
            print(rnn_datasets.shape)
            print(labels.shape)

            #print("cnn_dataset shape before reshape:", np.shape(cnn_datasets))
            # cnn_datasets = cnn_datasets.reshape(len(cnn_datasets), window_size, 9,9, 1)
            #print("cnn_dataset shape after reshape:", np.shape(cnn_datasets))
            one_hot_labels = np.array(list(pd.get_dummies(labels)))

            labels = np.asarray(pd.get_dummies(labels), dtype=np.int8)

            print(labels.shape)
            # shuffle data
            index = np.array(range(0, len(labels)))
            np.random.shuffle(index)

            cnn_datasets   = cnn_datasets[index]
            rnn_datasets   = rnn_datasets[index]
            labels  = labels[index]

            print(cnn_datasets.shape)
            print(rnn_datasets.shape)
            print(labels.shape)

            #print("**********(" + time.asctime(time.localtime(time.time())) + ") Load and Split dataset End **********\n")
            #print("**********(" + time.asctime(time.localtime(time.time())) + ") Define parameters and functions Begin: **********\n")
            print('cnn_datasets.shape,rnn_datasets.shape,labels.shape : ',cnn_datasets.shape,rnn_datasets.shape,labels.shape)
            #important
            cnn_datasets=cnn_datasets.reshape(2400,9,9,-1)# imp
            print(cnn_datasets.shape)
            print('cnn_datasets.shape : ',cnn_datasets.shape)







            print('========================Train / Test Shapes==============================')



            fold=10
            curr_fold=0
            #for curr_fold in range(fold): # kernel dies
                
            #   print('curr_fold / fold : ',curr_fold,' / ',fold)


            max_acc_acc_list=[]

            max_acc_loss_list=[]


            min_loss_acc_list=[]

            min_loss_loss_list=[]

            fold_size = cnn_datasets.shape[0]//fold
            indexes_list = [i for i in range(len(cnn_datasets))]
            indexes = np.array(indexes_list)
            split_list = [i for i in range(curr_fold*fold_size,(curr_fold+1)*fold_size)]
            split = np.array(split_list)

            cnn_test = cnn_datasets[split] 
            labels_test = labels[split]
            rnn_test = rnn_datasets[split]

            split = np.array(list(set(indexes_list)^set(split_list)))

            cnn_train = cnn_datasets[split]
            rnn_train = rnn_datasets[split]
            labels_train = labels[split]

            # train_sample = labels_train.shape[0]
            # print("training examples:", train_sample)
            # test_sample = labels_test.shape[0]
            # print("test examples    :",test_sample)
            print(cnn_test.shape,rnn_test.shape,cnn_train.shape,rnn_train.shape)

            print('================================ DNN ============================================')

            input_cnn=Input(shape=(9,9,128))

            print('input_cnn: ',input_cnn.shape)

            conv1=Conv2D(32,
                          kernel_size=(4,4),
                          strides=(1,1),
                          padding='same',
                          input_shape=(9,9,128)
                         )(input_cnn)


            bn1=BatchNormalization()(conv1)

            elu1=ELU()(bn1)

            print('elu1: ',elu1.shape)
            #?,9,9,32
            conv2=Conv2D(64,
                          kernel_size=(4,4),
                          strides=(1,1),
                          padding='same'
                         )(elu1)

            bn2=BatchNormalization()(conv2)

            elu2=ELU()(bn2)

            print('elu2: ',elu2.shape)
            #?,9,9,64

            conv3=Conv2D(128,
                          kernel_size=(4,4),
                          strides=(1,1),
                          padding='same'
                         )(elu2)

            bn3=BatchNormalization()(conv3)

            elu3=ELU()(bn3)


            print('elu3: ',elu3.shape)
            #?,9,9,128

            # mc.add(Flatten())
            # mc.add(Lambda(lambda x:x,output_shape=(9,9,32*4*128)))
            # mc.add(Lambda(K.reshape((-1,9,9,32*4*128))))
            reshape1=Reshape((9,9,-1))(elu3)


            print('reshape1: ',reshape1.shape)
            #?,9,9,32*4*128

            conv4=Conv2D(13,#32*4*128,
                          kernel_size=(1,1),
                          strides=(1,1),
                          padding='same'
                         )(reshape1)

            bn4=BatchNormalization()(conv4)

            elu4=ELU()(bn4)


            print('elu4: ',elu4.shape)
            #?,9,9,13 #32*4*128

            # mc.add(Flatten())

            # mc.add(Lambda(lambda x:x,output_shape=([13*9*9])))
            # mc.add(Lambda(K.reshape((None,13*9*9))))
            reshape2=Reshape(([13*9*9]))(elu4)


            print('reshape2: ',reshape2.shape)




            cnn_out_fuse=reshape2


            # cube=K.reshape(e3,(-1,9,9,32*4*128))#(e3)


            # flat=K.reshape(e4,(-1,13*9*9))#(e4) #1053

            # rnn_in=K.placeholder(shape=(None,128,32))
            # rnn_in=tf.convert_to_tensor(rnn_datasets,dtype='float32')


            # rnn_in.get_shape().as_list()

            # rnn_in=K.placeholder(shape=(None,128,32))
            # rnn_in_flat=K.reshape(rnn_in,[-1,32])

            # print('rnn_in ',rnn_in.shape)

            input_rnn=Input(shape=(128,32))
            print('input_rnn',input_rnn.shape)

            rnn_in_flat=Reshape((-1,32))(input_rnn)
            print('rnn_in_flat',rnn_in_flat.shape)
            # rnn_in_flat = Lambda(lambda x:x[:,0])(input_rnn)

            # rnn_fc_in1 =Dense(32)(rnn_in_flat)
            rnn_fc_in1 =Dense(1024)(rnn_in_flat)
            rnn_fc_in=ELU()(rnn_fc_in1)
            print('rnn_fc_in',rnn_fc_in.shape)

            # rnn_fc_in =Dense(1024)(input_rnn)

            # lstm_in=Reshape((-1,128,1024))(rnn_fc_in)
            lstm_in=Reshape((-1,1024))(rnn_fc_in)
            print('lstm_in',lstm_in.shape)

            cells=[]

            for i in range(2):
                cell=LSTMCell(32,unit_forget_bias=True,dropout=0.5)#'forget_bias'=1.0,'state_is_tuple'=True
                cells.append(cell)
            #     print(cell.shape)

            # lstm_cell=StackedRNNCells(cells)
            lstm_cell=RNN(cells)(lstm_in)
            # print(lstm_cell.shape)
            # op,states=RNN(cells)(lstm_in)
            print('lstm_cell',lstm_cell.shape)
            # output=K.transpose_shape((1,0,2),lstm_cell)
            # output=Permute((1,0,2))(lstm_cell)
            # output.reshape()
            output=lstm_cell
            print('output',output.shape)
            rnn_output=output[-1]
            # rnn_output

            print('rnn_output',rnn_output.shape)
            # shape_rnn_out=rnn_output.get_shape().as_list()
            lstm_fc_out=Dense(1024)(output)#shape_rnn_out[1]

            print('lstm_fc_out',lstm_fc_out.shape)


            # lstm_fc_out_2=Dense(1053)(lstm_fc_out)#shape_rnn_out[1]



            lstm_fc_drop=Dropout(0.5)(lstm_fc_out)
            # lstm_fc_drop
            print('lstm_fc_drop',lstm_fc_drop.shape)

            # fuse_cnn_rnn=add([cnn_out_fuse,lstm_fc_drop])

            fuse_cnn_rnn=concatenate([cnn_out_fuse,lstm_fc_drop])
            print('fuse_cnn_rnn ',fuse_cnn_rnn.shape)
            y=Dense(2,activation='softmax')(fuse_cnn_rnn) ## ,activity_regularizer=regularizers.l2(0.5)
            print(y.shape)
            y_pred=K.argmax(y,1)
            # y_pred=K.argmax(K.softmax(y))
            # y_posi=K.softmax(y)
            print('y ',y)


            directory_le007a='./lightningedge007a_results/'+with_or_without+'/'+arousal_or_valence+'/'+data_file
            try:
                if not os.path.exists(directory_le007a):
                    os.makedirs(directory_le007a)
            except OSError:
                print ('Error: Creating directory. ' +  directory_le007a)



            model=Model(inputs=[input_cnn,input_rnn],outputs=y)
            model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

            print('Model : ',with_or_without+'_'+arousal_or_valence+'_'+data_file+'_'+str(curr_fold)+'_fold')
            print(model.summary())

            m_val_acc=ModelCheckpoint('lightningedge007a_results/'+with_or_without+'/'+arousal_or_valence+'/'+data_file+'/'+'max_acc_'+with_or_without+'_'+arousal_or_valence+'_'+data_file+'.h5',monitor='val_accuracy',mode='max',verbose=1,save_best_only=True)
            m_val_loss=ModelCheckpoint('lightningedge007a_results/'+with_or_without+'/'+arousal_or_valence+'/'+data_file+'/'+'min_loss_'+with_or_without+'_'+arousal_or_valence+'_'+data_file+'.h5',monitor='val_loss',mode='min',verbose=1,save_best_only=True)


            tb_log_dir='lightningedge007a_results\\'+with_or_without+'\\'+arousal_or_valence+'\\'+data_file+'\\'+'logs_'+with_or_without+'_'+arousal_or_valence+'_'+data_file
            #tb_log_dir='lightningedge007a_results/'+with_or_without+'/'+arousal_or_valence+'/'+data_file+'/'+'logs_'+with_or_without+'_'+arousal_or_valence+'_'+data_file

            createFolder(tb_log_dir)


            #log_dir='lightningedge007a_results\\'+with_or_without+'\\'+arousal_or_valence+'\\'+data_file+'\\'+'logs_'+with_or_without+'_'+arousal_or_valence+'_'+data_file+'_'+str(curr_fold)+'_fold\\' #datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
            tensorboard_callback = callbacks.TensorBoard(log_dir=tb_log_dir)#, histogram_freq=1)

            plotpicture=plot_model(model, to_file='lightningedge007a_results/'+with_or_without+'/'+arousal_or_valence+'/'+data_file+'/'+'model_'+with_or_without+'_'+arousal_or_valence+'_'+data_file+'.png', show_shapes=True)



            history=model.fit([cnn_train,rnn_train],labels_train,batch_size=128,epochs=50,callbacks=[tensorboard_callback,m_val_acc,m_val_loss],validation_split=0.2)


            #Plot values
            plt.plot(history.history['accuracy'])
            plt.plot(history.history['val_accuracy'])
            plt.title('max_acc_'+with_or_without+'_'+arousal_or_valence+'_'+data_file)
            plt.ylabel('Accuracy')
            plt.xlabel('Epoch')
            plt.legend(['Train', 'Validation'], loc='upper right',bbox_to_anchor=(1.3,1))
            plt.savefig('lightningedge007a_results/'+with_or_without+'/'+arousal_or_valence+'/'+data_file+'/'+'max_acc_'+with_or_without+'_'+arousal_or_valence+'_'+data_file+'.png',bbox_inches='tight')
            #plt.show()
            plt.close()


            # Plot training & validation loss values
            plt.plot(history.history['loss'])
            plt.plot(history.history['val_loss'])
            plt.title('min_loss_'+with_or_without+'_'+arousal_or_valence+'_'+data_file+'_'+str(curr_fold)+'_fold')
            plt.ylabel('Loss')
            plt.xlabel('Epoch')
            plt.legend(['Train', 'Validation'], loc='upper right',bbox_to_anchor=(1.3,1))
            plt.savefig('lightningedge007a_results/'+with_or_without+'/'+arousal_or_valence+'/'+data_file+'/'+'min_loss_'+with_or_without+'_'+arousal_or_valence+'_'+data_file+'.png',bbox_inches='tight')
            #plt.show()
            plt.close()

            vam=load_model('lightningedge007a_results/'+with_or_without+'/'+arousal_or_valence+'/'+data_file+'/'+'max_acc_'+with_or_without+'_'+arousal_or_valence+'_'+data_file+'.h5')
            vlm=load_model('lightningedge007a_results/'+with_or_without+'/'+arousal_or_valence+'/'+data_file+'/'+'min_loss_'+with_or_without+'_'+arousal_or_valence+'_'+data_file+'.h5')
            # pred_labels=vam()
            vam_eval=vam.evaluate([cnn_test,rnn_test],labels_test)
            print('max acc: ',vam_eval)
            vlm_eval=vlm.evaluate([cnn_test,rnn_test],labels_test)
            print('min loss: ',vlm_eval)


            max_acc_acc_list.append(vam_eval[1])

            max_acc_loss_list.append(vam_eval[0])


            min_loss_acc_list.append(vlm_eval[1])

            min_loss_loss_list.append(vlm_eval[0])

            pickle.dump(max_acc_acc_list,open('lightningedge007a_results/'+with_or_without+'/'+arousal_or_valence+'/'+data_file+'/'+'max_acc_acc_list_'+with_or_without+'_'+arousal_or_valence+'_'+data_file+'.pkl','wb'))

            pickle.dump(max_acc_loss_list,open('lightningedge007a_results/'+with_or_without+'/'+arousal_or_valence+'/'+data_file+'/'+'max_acc_loss_list_'+with_or_without+'_'+arousal_or_valence+'_'+data_file+'.pkl','wb'))


            pickle.dump(min_loss_acc_list,open('lightningedge007a_results/'+with_or_without+'/'+arousal_or_valence+'/'+data_file+'/'+'min_loss_acc_list_'+with_or_without+'_'+arousal_or_valence+'_'+data_file+'.pkl','wb'))

            pickle.dump(min_loss_loss_list,open('lightningedge007a_results/'+with_or_without+'/'+arousal_or_valence+'/'+data_file+'/'+'min_loss_loss_list_'+with_or_without+'_'+arousal_or_valence+'_'+data_file+'.pkl','wb'))



---------------------------------------------------
yes arousal s06
---------------------------------------------------

loaded shape: (2400,)
(2400, 128, 9, 9)
(2400, 128, 32)
(2400,)
(2400, 2)
(2400, 128, 9, 9)
(2400, 128, 32)
(2400, 2)
cnn_datasets.shape,rnn_datasets.shape,labels.shape :  (2400, 128, 9, 9) (2400, 128, 32) (2400, 2)
(2400, 9, 9, 128)
cnn_datasets.shape :  (2400, 9, 9, 128)
========================Train / Test Shapes==============================
(240, 9, 9, 128) (240, 128, 32) (2160, 9, 9, 128) (2160, 128, 32)
================================ DNN ============================================
input_cnn:  (None, 9, 9, 128)
elu1:  (None, 9, 9, 32)
elu2:  (None, 9, 9, 64)
elu3:  (None, 9, 9, 128)
reshape1:  (None, 9, 9, None)
elu4:  (None, 9, 9, 13)
reshape2:  (None, 1053)
input_rnn (None, 128, 32)
rnn_in_flat (None, None, 32)
rnn_fc_in (None, 128, 1024)
lstm_in (None, None, 1024)
lstm_cell (None, 32)
output (None, 32)
rnn_output (32,)
lstm_fc_out (None, 1024)
lstm_fc_dro

C:\Users\ABHISHEK_VERMA\Anaconda3\lib\site-packages\keras\callbacks\callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (10.110810). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


1728/1728 [==============================] - 28s 16ms/step - loss: 0.3893 - accuracy: 0.8281 - val_loss: 0.2382 - val_accuracy: 0.9074

Epoch 00001: val_accuracy improved from -inf to 0.90741, saving model to lightningedge007a_results/yes/arousal/s06/max_acc_yes_arousal_s06.h5

Epoch 00001: val_loss improved from inf to 0.23818, saving model to lightningedge007a_results/yes/arousal/s06/min_loss_yes_arousal_s06.h5
Epoch 2/50
1728/1728 [==============================] - 3s 2ms/step - loss: 0.0796 - accuracy: 0.9757 - val_loss: 0.1671 - val_accuracy: 0.9352

Epoch 00002: val_accuracy improved from 0.90741 to 0.93519, saving model to lightningedge007a_results/yes/arousal/s06/max_acc_yes_arousal_s06.h5

Epoch 00002: val_loss improved from 0.23818 to 0.16711, saving model to lightningedge007a_results/yes/arousal/s06/min_loss_yes_arousal_s06.h5
Epoch 3/50
1728/1728 [==============================] - 3s 2ms/step - loss: 0.0224 - accuracy: 0.9965 - val_loss: 0.0955 - val_accuracy: 0.9722

Epoch

1728/1728 [==============================] - 3s 2ms/step - loss: 2.5064e-04 - accuracy: 1.0000 - val_loss: 0.0674 - val_accuracy: 0.9815

Epoch 00024: val_accuracy did not improve from 0.98148

Epoch 00024: val_loss did not improve from 0.06723
Epoch 25/50
1728/1728 [==============================] - 3s 2ms/step - loss: 2.2748e-04 - accuracy: 1.0000 - val_loss: 0.0675 - val_accuracy: 0.9815

Epoch 00025: val_accuracy did not improve from 0.98148

Epoch 00025: val_loss did not improve from 0.06723
Epoch 26/50
1728/1728 [==============================] - 3s 2ms/step - loss: 2.1617e-04 - accuracy: 1.0000 - val_loss: 0.0678 - val_accuracy: 0.9815

Epoch 00026: val_accuracy did not improve from 0.98148

Epoch 00026: val_loss did not improve from 0.06723
Epoch 27/50
1728/1728 [==============================] - 3s 2ms/step - loss: 2.0250e-04 - accuracy: 1.0000 - val_loss: 0.0680 - val_accuracy: 0.9815

Epoch 00027: val_accuracy did not improve from 0.98148

Epoch 00027: val_loss did not impro

Train on 1728 samples, validate on 432 samples
Epoch 1/50
 256/1728 [===>..........................] - ETA: 10s - loss: 0.7190 - accuracy: 0.6172

...

1728/1728 [==============================] - 3s 2ms/step - loss: 4.0668e-04 - accuracy: 1.0000 - val_loss: 0.0684 - val_accuracy: 0.9745

Epoch 00024: val_accuracy did not improve from 0.97454

Epoch 00024: val_loss improved from 0.06871 to 0.06842, saving model to lightningedge007a_results/yes/arousal/s09/min_loss_yes_arousal_s09.h5
Epoch 25/50
1728/1728 [==============================] - 3s 2ms/step - loss: 3.7680e-04 - accuracy: 1.0000 - val_loss: 0.0683 - val_accuracy: 0.9745

Epoch 00025: val_accuracy did not improve from 0.97454

Epoch 00025: val_loss improved from 0.06842 to 0.06834, saving model to lightningedge007a_results/yes/arousal/s09/min_loss_yes_arousal_s09.h5
Epoch 26/50
1728/1728 [==============================] - 3s 2ms/step - loss: 3.5452e-04 - accuracy: 1.0000 - val_loss: 0.0680 - val_accuracy: 0.9745

Epoch 00026: val_accuracy did not improve from 0.97454

Epoch 00026: val_loss improved from 0.06834 to 0.06800, saving model to lightningedge007a_results/yes/arousal/

(240, 9, 9, 128) (240, 128, 32) (2160, 9, 9, 128) (2160, 128, 32)
================================ DNN ============================================
input_cnn:  (None, 9, 9, 128)
elu1:  (None, 9, 9, 32)
elu2:  (None, 9, 9, 64)
elu3:  (None, 9, 9, 128)
reshape1:  (None, 9, 9, None)
elu4:  (None, 9, 9, 13)
reshape2:  (None, 1053)
input_rnn (None, 128, 32)
rnn_in_flat (None, None, 32)
rnn_fc_in (None, 128, 1024)
lstm_in (None, None, 1024)
lstm_cell (None, 32)
output (None, 32)
rnn_output (32,)
lstm_fc_out (None, 1024)
lstm_fc_drop (None, 1024)
fuse_cnn_rnn  (None, 2077)
(None, 2)
y  Tensor("dense_15/Softmax:0", shape=(None, 2), dtype=float32)
Model :  yes_arousal_s10_0_fold
Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            (None, 9, 9, 128)    0                                            
_______

C:\Users\ABHISHEK_VERMA\Anaconda3\lib\site-packages\keras\callbacks\callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (14.314133). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


1728/1728 [==============================] - 33s 19ms/step - loss: 0.4324 - accuracy: 0.8166 - val_loss: 0.2397 - val_accuracy: 0.9051

Epoch 00001: val_accuracy improved from -inf to 0.90509, saving model to lightningedge007a_results/yes/arousal/s10/max_acc_yes_arousal_s10.h5

Epoch 00001: val_loss improved from inf to 0.23969, saving model to lightningedge007a_results/yes/arousal/s10/min_loss_yes_arousal_s10.h5
Epoch 2/50
1728/1728 [==============================] - 3s 2ms/step - loss: 0.0885 - accuracy: 0.9682 - val_loss: 0.1599 - val_accuracy: 0.9398

Epoch 00002: val_accuracy improved from 0.90509 to 0.93981, saving model to lightningedge007a_results/yes/arousal/s10/max_acc_yes_arousal_s10.h5

Epoch 00002: val_loss improved from 0.23969 to 0.15988, saving model to lightningedge007a_results/yes/arousal/s10/min_loss_yes_arousal_s10.h5
Epoch 3/50
1728/1728 [==============================] - 3s 2ms/step - loss: 0.0336 - accuracy: 0.9896 - val_loss: 0.0950 - val_accuracy: 0.9676

Epoch


Epoch 00027: val_accuracy did not improve from 0.97685

Epoch 00027: val_loss did not improve from 0.06532
Epoch 28/50
1728/1728 [==============================] - 3s 2ms/step - loss: 2.0049e-04 - accuracy: 1.0000 - val_loss: 0.0715 - val_accuracy: 0.9769

Epoch 00028: val_accuracy did not improve from 0.97685

Epoch 00028: val_loss did not improve from 0.06532
Epoch 29/50
1728/1728 [==============================] - 3s 2ms/step - loss: 1.9157e-04 - accuracy: 1.0000 - val_loss: 0.0716 - val_accuracy: 0.9769

Epoch 00029: val_accuracy did not improve from 0.97685

Epoch 00029: val_loss did not improve from 0.06532
Epoch 30/50
1728/1728 [==============================] - 3s 2ms/step - loss: 1.8054e-04 - accuracy: 1.0000 - val_loss: 0.0713 - val_accuracy: 0.9792

Epoch 00030: val_accuracy improved from 0.97685 to 0.97917, saving model to lightningedge007a_results/yes/arousal/s10/max_acc_yes_arousal_s10.h5

Epoch 00030: val_loss did not improve from 0.06532
Epoch 31/50
1728/1728 [========

In [11]:
gc.collect()

229356

In [10]:
import gc

In [11]:
data_files=data_files[10:15]
data_files

['s11', 's12', 's13', 's14', 's15']

In [12]:
np.random.seed(32)

window_size = 128

cnn_suffix ='.mat_win_128_cnn_dataset.pkl'
rnn_suffix ='.mat_win_128_rnn_dataset.pkl'
label_suffix ='.mat_win_128_labels.pkl'

for with_or_without in baseline_preprocessing:
    for arousal_or_valence in emotions:
        for data_file in data_files:
            
            gc.collect()
            
            print('---------------------------------------------------')
            print(with_or_without+' '+arousal_or_valence+' '+data_file)
            print('---------------------------------------------------\n')

            #data_file    ='s17'
            #arousal_or_valence = 'valence'
            #with_or_without = 'yes'

            dataset_dir = 'deap_shuffled_data/'+with_or_without+'_'+arousal_or_valence+'/'
            ###load training set
            with open(dataset_dir + data_file + cnn_suffix, "rb") as fp:
                cnn_datasets = pickle.load(fp)
            with open(dataset_dir + data_file + rnn_suffix, "rb") as fp:
                rnn_datasets = pickle.load(fp)
            with open(dataset_dir + data_file + label_suffix, "rb") as fp:
                labels = pickle.load(fp)
                labels = np.transpose(labels)
                print("loaded shape:",labels.shape)
            lables_backup = labels

            print(cnn_datasets.shape)
            print(rnn_datasets.shape)
            print(labels.shape)

            #print("cnn_dataset shape before reshape:", np.shape(cnn_datasets))
            # cnn_datasets = cnn_datasets.reshape(len(cnn_datasets), window_size, 9,9, 1)
            #print("cnn_dataset shape after reshape:", np.shape(cnn_datasets))
            one_hot_labels = np.array(list(pd.get_dummies(labels)))

            labels = np.asarray(pd.get_dummies(labels), dtype=np.int8)

            print(labels.shape)
            # shuffle data
            index = np.array(range(0, len(labels)))
            np.random.shuffle(index)

            cnn_datasets   = cnn_datasets[index]
            rnn_datasets   = rnn_datasets[index]
            labels  = labels[index]

            print(cnn_datasets.shape)
            print(rnn_datasets.shape)
            print(labels.shape)

            #print("**********(" + time.asctime(time.localtime(time.time())) + ") Load and Split dataset End **********\n")
            #print("**********(" + time.asctime(time.localtime(time.time())) + ") Define parameters and functions Begin: **********\n")
            print('cnn_datasets.shape,rnn_datasets.shape,labels.shape : ',cnn_datasets.shape,rnn_datasets.shape,labels.shape)
            #important
            cnn_datasets=cnn_datasets.reshape(2400,9,9,-1)# imp
            print(cnn_datasets.shape)
            print('cnn_datasets.shape : ',cnn_datasets.shape)







            print('========================Train / Test Shapes==============================')



            fold=10
            curr_fold=0
            #for curr_fold in range(fold): # kernel dies
                
            #   print('curr_fold / fold : ',curr_fold,' / ',fold)


            max_acc_acc_list=[]

            max_acc_loss_list=[]


            min_loss_acc_list=[]

            min_loss_loss_list=[]

            fold_size = cnn_datasets.shape[0]//fold
            indexes_list = [i for i in range(len(cnn_datasets))]
            indexes = np.array(indexes_list)
            split_list = [i for i in range(curr_fold*fold_size,(curr_fold+1)*fold_size)]
            split = np.array(split_list)

            cnn_test = cnn_datasets[split] 
            labels_test = labels[split]
            rnn_test = rnn_datasets[split]

            split = np.array(list(set(indexes_list)^set(split_list)))

            cnn_train = cnn_datasets[split]
            rnn_train = rnn_datasets[split]
            labels_train = labels[split]

            # train_sample = labels_train.shape[0]
            # print("training examples:", train_sample)
            # test_sample = labels_test.shape[0]
            # print("test examples    :",test_sample)
            print(cnn_test.shape,rnn_test.shape,cnn_train.shape,rnn_train.shape)

            print('================================ DNN ============================================')

            input_cnn=Input(shape=(9,9,128))

            print('input_cnn: ',input_cnn.shape)

            conv1=Conv2D(32,
                          kernel_size=(4,4),
                          strides=(1,1),
                          padding='same',
                          input_shape=(9,9,128)
                         )(input_cnn)


            bn1=BatchNormalization()(conv1)

            elu1=ELU()(bn1)

            print('elu1: ',elu1.shape)
            #?,9,9,32
            conv2=Conv2D(64,
                          kernel_size=(4,4),
                          strides=(1,1),
                          padding='same'
                         )(elu1)

            bn2=BatchNormalization()(conv2)

            elu2=ELU()(bn2)

            print('elu2: ',elu2.shape)
            #?,9,9,64

            conv3=Conv2D(128,
                          kernel_size=(4,4),
                          strides=(1,1),
                          padding='same'
                         )(elu2)

            bn3=BatchNormalization()(conv3)

            elu3=ELU()(bn3)


            print('elu3: ',elu3.shape)
            #?,9,9,128

            # mc.add(Flatten())
            # mc.add(Lambda(lambda x:x,output_shape=(9,9,32*4*128)))
            # mc.add(Lambda(K.reshape((-1,9,9,32*4*128))))
            reshape1=Reshape((9,9,-1))(elu3)


            print('reshape1: ',reshape1.shape)
            #?,9,9,32*4*128

            conv4=Conv2D(13,#32*4*128,
                          kernel_size=(1,1),
                          strides=(1,1),
                          padding='same'
                         )(reshape1)

            bn4=BatchNormalization()(conv4)

            elu4=ELU()(bn4)


            print('elu4: ',elu4.shape)
            #?,9,9,13 #32*4*128

            # mc.add(Flatten())

            # mc.add(Lambda(lambda x:x,output_shape=([13*9*9])))
            # mc.add(Lambda(K.reshape((None,13*9*9))))
            reshape2=Reshape(([13*9*9]))(elu4)


            print('reshape2: ',reshape2.shape)




            cnn_out_fuse=reshape2


            # cube=K.reshape(e3,(-1,9,9,32*4*128))#(e3)


            # flat=K.reshape(e4,(-1,13*9*9))#(e4) #1053

            # rnn_in=K.placeholder(shape=(None,128,32))
            # rnn_in=tf.convert_to_tensor(rnn_datasets,dtype='float32')


            # rnn_in.get_shape().as_list()

            # rnn_in=K.placeholder(shape=(None,128,32))
            # rnn_in_flat=K.reshape(rnn_in,[-1,32])

            # print('rnn_in ',rnn_in.shape)

            input_rnn=Input(shape=(128,32))
            print('input_rnn',input_rnn.shape)

            rnn_in_flat=Reshape((-1,32))(input_rnn)
            print('rnn_in_flat',rnn_in_flat.shape)
            # rnn_in_flat = Lambda(lambda x:x[:,0])(input_rnn)

            # rnn_fc_in1 =Dense(32)(rnn_in_flat)
            rnn_fc_in1 =Dense(1024)(rnn_in_flat)
            rnn_fc_in=ELU()(rnn_fc_in1)
            print('rnn_fc_in',rnn_fc_in.shape)

            # rnn_fc_in =Dense(1024)(input_rnn)

            # lstm_in=Reshape((-1,128,1024))(rnn_fc_in)
            lstm_in=Reshape((-1,1024))(rnn_fc_in)
            print('lstm_in',lstm_in.shape)

            cells=[]

            for i in range(2):
                cell=LSTMCell(32,unit_forget_bias=True,dropout=0.5)#'forget_bias'=1.0,'state_is_tuple'=True
                cells.append(cell)
            #     print(cell.shape)

            # lstm_cell=StackedRNNCells(cells)
            lstm_cell=RNN(cells)(lstm_in)
            # print(lstm_cell.shape)
            # op,states=RNN(cells)(lstm_in)
            print('lstm_cell',lstm_cell.shape)
            # output=K.transpose_shape((1,0,2),lstm_cell)
            # output=Permute((1,0,2))(lstm_cell)
            # output.reshape()
            output=lstm_cell
            print('output',output.shape)
            rnn_output=output[-1]
            # rnn_output

            print('rnn_output',rnn_output.shape)
            # shape_rnn_out=rnn_output.get_shape().as_list()
            lstm_fc_out=Dense(1024)(output)#shape_rnn_out[1]

            print('lstm_fc_out',lstm_fc_out.shape)


            # lstm_fc_out_2=Dense(1053)(lstm_fc_out)#shape_rnn_out[1]



            lstm_fc_drop=Dropout(0.5)(lstm_fc_out)
            # lstm_fc_drop
            print('lstm_fc_drop',lstm_fc_drop.shape)

            # fuse_cnn_rnn=add([cnn_out_fuse,lstm_fc_drop])

            fuse_cnn_rnn=concatenate([cnn_out_fuse,lstm_fc_drop])
            print('fuse_cnn_rnn ',fuse_cnn_rnn.shape)
            y=Dense(2,activation='softmax')(fuse_cnn_rnn) ## ,activity_regularizer=regularizers.l2(0.5)
            print(y.shape)
            y_pred=K.argmax(y,1)
            # y_pred=K.argmax(K.softmax(y))
            # y_posi=K.softmax(y)
            print('y ',y)


            directory_le007a='./lightningedge007a_results/'+with_or_without+'/'+arousal_or_valence+'/'+data_file
            try:
                if not os.path.exists(directory_le007a):
                    os.makedirs(directory_le007a)
            except OSError:
                print ('Error: Creating directory. ' +  directory_le007a)



            model=Model(inputs=[input_cnn,input_rnn],outputs=y)
            model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

            print('Model : ',with_or_without+'_'+arousal_or_valence+'_'+data_file+'_'+str(curr_fold)+'_fold')
            print(model.summary())

            m_val_acc=ModelCheckpoint('lightningedge007a_results/'+with_or_without+'/'+arousal_or_valence+'/'+data_file+'/'+'max_acc_'+with_or_without+'_'+arousal_or_valence+'_'+data_file+'.h5',monitor='val_accuracy',mode='max',verbose=1,save_best_only=True)
            m_val_loss=ModelCheckpoint('lightningedge007a_results/'+with_or_without+'/'+arousal_or_valence+'/'+data_file+'/'+'min_loss_'+with_or_without+'_'+arousal_or_valence+'_'+data_file+'.h5',monitor='val_loss',mode='min',verbose=1,save_best_only=True)


            tb_log_dir='lightningedge007a_results\\'+with_or_without+'\\'+arousal_or_valence+'\\'+data_file+'\\'+'logs_'+with_or_without+'_'+arousal_or_valence+'_'+data_file
            #tb_log_dir='lightningedge007a_results/'+with_or_without+'/'+arousal_or_valence+'/'+data_file+'/'+'logs_'+with_or_without+'_'+arousal_or_valence+'_'+data_file

            createFolder(tb_log_dir)


            #log_dir='lightningedge007a_results\\'+with_or_without+'\\'+arousal_or_valence+'\\'+data_file+'\\'+'logs_'+with_or_without+'_'+arousal_or_valence+'_'+data_file+'_'+str(curr_fold)+'_fold\\' #datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
            tensorboard_callback = callbacks.TensorBoard(log_dir=tb_log_dir)#, histogram_freq=1)

            plotpicture=plot_model(model, to_file='lightningedge007a_results/'+with_or_without+'/'+arousal_or_valence+'/'+data_file+'/'+'model_'+with_or_without+'_'+arousal_or_valence+'_'+data_file+'.png', show_shapes=True)

            
            
            gc.collect()
            
            

            history=model.fit([cnn_train,rnn_train],labels_train,batch_size=128,epochs=50,callbacks=[tensorboard_callback,m_val_acc,m_val_loss],validation_split=0.2)


            #Plot values
            plt.plot(history.history['accuracy'])
            plt.plot(history.history['val_accuracy'])
            plt.title('max_acc_'+with_or_without+'_'+arousal_or_valence+'_'+data_file)
            plt.ylabel('Accuracy')
            plt.xlabel('Epoch')
            plt.legend(['Train', 'Validation'], loc='upper right',bbox_to_anchor=(1.3,1))
            plt.savefig('lightningedge007a_results/'+with_or_without+'/'+arousal_or_valence+'/'+data_file+'/'+'max_acc_'+with_or_without+'_'+arousal_or_valence+'_'+data_file+'.png',bbox_inches='tight')
            #plt.show()
            plt.close()


            # Plot training & validation loss values
            plt.plot(history.history['loss'])
            plt.plot(history.history['val_loss'])
            plt.title('min_loss_'+with_or_without+'_'+arousal_or_valence+'_'+data_file+'_'+str(curr_fold)+'_fold')
            plt.ylabel('Loss')
            plt.xlabel('Epoch')
            plt.legend(['Train', 'Validation'], loc='upper right',bbox_to_anchor=(1.3,1))
            plt.savefig('lightningedge007a_results/'+with_or_without+'/'+arousal_or_valence+'/'+data_file+'/'+'min_loss_'+with_or_without+'_'+arousal_or_valence+'_'+data_file+'.png',bbox_inches='tight')
            #plt.show()
            plt.close()

            vam=load_model('lightningedge007a_results/'+with_or_without+'/'+arousal_or_valence+'/'+data_file+'/'+'max_acc_'+with_or_without+'_'+arousal_or_valence+'_'+data_file+'.h5')
            vlm=load_model('lightningedge007a_results/'+with_or_without+'/'+arousal_or_valence+'/'+data_file+'/'+'min_loss_'+with_or_without+'_'+arousal_or_valence+'_'+data_file+'.h5')
            # pred_labels=vam()
            vam_eval=vam.evaluate([cnn_test,rnn_test],labels_test)
            print('max acc: ',vam_eval)
            vlm_eval=vlm.evaluate([cnn_test,rnn_test],labels_test)
            print('min loss: ',vlm_eval)


            max_acc_acc_list.append(vam_eval[1])

            max_acc_loss_list.append(vam_eval[0])


            min_loss_acc_list.append(vlm_eval[1])

            min_loss_loss_list.append(vlm_eval[0])

            pickle.dump(max_acc_acc_list,open('lightningedge007a_results/'+with_or_without+'/'+arousal_or_valence+'/'+data_file+'/'+'max_acc_acc_list_'+with_or_without+'_'+arousal_or_valence+'_'+data_file+'.pkl','wb'))

            pickle.dump(max_acc_loss_list,open('lightningedge007a_results/'+with_or_without+'/'+arousal_or_valence+'/'+data_file+'/'+'max_acc_loss_list_'+with_or_without+'_'+arousal_or_valence+'_'+data_file+'.pkl','wb'))


            pickle.dump(min_loss_acc_list,open('lightningedge007a_results/'+with_or_without+'/'+arousal_or_valence+'/'+data_file+'/'+'min_loss_acc_list_'+with_or_without+'_'+arousal_or_valence+'_'+data_file+'.pkl','wb'))

            pickle.dump(min_loss_loss_list,open('lightningedge007a_results/'+with_or_without+'/'+arousal_or_valence+'/'+data_file+'/'+'min_loss_loss_list_'+with_or_without+'_'+arousal_or_valence+'_'+data_file+'.pkl','wb'))
            
            
            gc.collect()


---------------------------------------------------
yes arousal s11
---------------------------------------------------

loaded shape: (2400,)
(2400, 128, 9, 9)
(2400, 128, 32)
(2400,)
(2400, 2)
(2400, 128, 9, 9)
(2400, 128, 32)
(2400, 2)
cnn_datasets.shape,rnn_datasets.shape,labels.shape :  (2400, 128, 9, 9) (2400, 128, 32) (2400, 2)
(2400, 9, 9, 128)
cnn_datasets.shape :  (2400, 9, 9, 128)
========================Train / Test Shapes==============================
(240, 9, 9, 128) (240, 128, 32) (2160, 9, 9, 128) (2160, 128, 32)
================================ DNN ============================================
input_cnn:  (None, 9, 9, 128)
elu1:  (None, 9, 9, 32)
elu2:  (None, 9, 9, 64)
elu3:  (None, 9, 9, 128)
reshape1:  (None, 9, 9, None)
elu4:  (None, 9, 9, 13)
reshape2:  (None, 1053)
input_rnn (None, 128, 32)
rnn_in_flat (None, None, 32)
rnn_fc_in (None, 128, 1024)
lstm_in (None, None, 1024)
lstm_cell (None, 32)
output (None, 32)
rnn_output (32,)
lstm_fc_out (None, 1024)
lstm_fc_dro

C:\Users\ABHISHEK_VERMA\Anaconda3\lib\site-packages\keras\callbacks\callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (10.125877). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


1728/1728 [==============================] - 27s 16ms/step - loss: 0.5811 - accuracy: 0.7407 - val_loss: 0.3863 - val_accuracy: 0.8079

Epoch 00001: val_accuracy improved from -inf to 0.80787, saving model to lightningedge007a_results/yes/arousal/s11/max_acc_yes_arousal_s11.h5

Epoch 00001: val_loss improved from inf to 0.38629, saving model to lightningedge007a_results/yes/arousal/s11/min_loss_yes_arousal_s11.h5
Epoch 2/50
1728/1728 [==============================] - 3s 2ms/step - loss: 0.2222 - accuracy: 0.9109 - val_loss: 0.2988 - val_accuracy: 0.8657

Epoch 00002: val_accuracy improved from 0.80787 to 0.86574, saving model to lightningedge007a_results/yes/arousal/s11/max_acc_yes_arousal_s11.h5

Epoch 00002: val_loss improved from 0.38629 to 0.29885, saving model to lightningedge007a_results/yes/arousal/s11/min_loss_yes_arousal_s11.h5
Epoch 3/50
1728/1728 [==============================] - 3s 2ms/step - loss: 0.1008 - accuracy: 0.9716 - val_loss: 0.2378 - val_accuracy: 0.9120

Epoch

1728/1728 [==============================] - 3s 2ms/step - loss: 4.4561e-04 - accuracy: 1.0000 - val_loss: 0.1199 - val_accuracy: 0.9583

Epoch 00026: val_accuracy did not improve from 0.96065

Epoch 00026: val_loss did not improve from 0.11830
Epoch 27/50
1728/1728 [==============================] - 3s 2ms/step - loss: 4.1298e-04 - accuracy: 1.0000 - val_loss: 0.1208 - val_accuracy: 0.9583

Epoch 00027: val_accuracy did not improve from 0.96065

Epoch 00027: val_loss did not improve from 0.11830
Epoch 28/50
1728/1728 [==============================] - 3s 2ms/step - loss: 3.8847e-04 - accuracy: 1.0000 - val_loss: 0.1204 - val_accuracy: 0.9606

Epoch 00028: val_accuracy did not improve from 0.96065

Epoch 00028: val_loss did not improve from 0.11830
Epoch 29/50
1728/1728 [==============================] - 3s 2ms/step - loss: 3.6219e-04 - accuracy: 1.0000 - val_loss: 0.1198 - val_accuracy: 0.9583

Epoch 00029: val_accuracy did not improve from 0.96065

Epoch 00029: val_loss did not impro

Train on 1728 samples, validate on 432 samples
Epoch 1/50
 256/1728 [===>..........................] - ETA: 10s - loss: 0.5615 - accuracy: 0.7031

...

1728/1728 [==============================] - 3s 2ms/step - loss: 2.4475e-04 - accuracy: 1.0000 - val_loss: 0.0960 - val_accuracy: 0.9676

Epoch 00027: val_accuracy did not improve from 0.96991

Epoch 00027: val_loss did not improve from 0.09234
Epoch 28/50
1728/1728 [==============================] - 3s 2ms/step - loss: 2.2945e-04 - accuracy: 1.0000 - val_loss: 0.0958 - val_accuracy: 0.9676

Epoch 00028: val_accuracy did not improve from 0.96991

Epoch 00028: val_loss did not improve from 0.09234
Epoch 29/50
1728/1728 [==============================] - 3s 2ms/step - loss: 2.1860e-04 - accuracy: 1.0000 - val_loss: 0.0957 - val_accuracy: 0.9722

Epoch 00029: val_accuracy improved from 0.96991 to 0.97222, saving model to lightningedge007a_results/yes/arousal/s14/max_acc_yes_arousal_s14.h5

Epoch 00029: val_loss did not improve from 0.09234
Epoch 30/50
1728/1728 [==============================] - 3s 2ms/step - loss: 1.9900e-04 - accuracy: 1.0000 - val_loss: 0.0959 - val_accuracy: 0.9699

E

Train on 1728 samples, validate on 432 samples
Epoch 1/50
 256/1728 [===>..........................] - ETA: 11s - loss: 0.6608 - accuracy: 0.6875

C:\Users\ABHISHEK_VERMA\Anaconda3\lib\site-packages\keras\callbacks\callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (10.250128). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


1728/1728 [==============================] - 25s 15ms/step - loss: 0.2600 - accuracy: 0.8924 - val_loss: 0.1596 - val_accuracy: 0.9329

Epoch 00001: val_accuracy improved from -inf to 0.93287, saving model to lightningedge007a_results/yes/arousal/s15/max_acc_yes_arousal_s15.h5

Epoch 00001: val_loss improved from inf to 0.15957, saving model to lightningedge007a_results/yes/arousal/s15/min_loss_yes_arousal_s15.h5
Epoch 2/50
1728/1728 [==============================] - 3s 2ms/step - loss: 0.0400 - accuracy: 0.9850 - val_loss: 0.0934 - val_accuracy: 0.9653

Epoch 00002: val_accuracy improved from 0.93287 to 0.96528, saving model to lightningedge007a_results/yes/arousal/s15/max_acc_yes_arousal_s15.h5

Epoch 00002: val_loss improved from 0.15957 to 0.09338, saving model to lightningedge007a_results/yes/arousal/s15/min_loss_yes_arousal_s15.h5
Epoch 3/50
1728/1728 [==============================] - 3s 2ms/step - loss: 0.0131 - accuracy: 0.9977 - val_loss: 0.0723 - val_accuracy: 0.9792

Epoch


Epoch 00023: val_accuracy did not improve from 0.99074

Epoch 00023: val_loss improved from 0.02765 to 0.02763, saving model to lightningedge007a_results/yes/arousal/s15/min_loss_yes_arousal_s15.h5
Epoch 24/50
1728/1728 [==============================] - 3s 2ms/step - loss: 1.2440e-04 - accuracy: 1.0000 - val_loss: 0.0278 - val_accuracy: 0.9907

Epoch 00024: val_accuracy did not improve from 0.99074

Epoch 00024: val_loss did not improve from 0.02763
Epoch 25/50
1728/1728 [==============================] - 3s 2ms/step - loss: 1.1879e-04 - accuracy: 1.0000 - val_loss: 0.0278 - val_accuracy: 0.9907

Epoch 00025: val_accuracy did not improve from 0.99074

Epoch 00025: val_loss did not improve from 0.02763
Epoch 26/50
1728/1728 [==============================] - 3s 2ms/step - loss: 1.1033e-04 - accuracy: 1.0000 - val_loss: 0.0278 - val_accuracy: 0.9907

Epoch 00026: val_accuracy did not improve from 0.99074

Epoch 00026: val_loss did not improve from 0.02763
Epoch 27/50
1728/1728 [=======

In [15]:
vam

In [18]:
gc.collect()

20